In [ ]:
import os
import uuid
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import ChatMessageContent, FunctionCallContent, FunctionResultContent
from semantic_kernel.core_plugins import SessionsPythonTool
from semantic_kernel.core_plugins.sessions_python_tool import SessionsPythonSettings, SessionsPythonSettings
from azure.identity import EnvironmentCredential, get_bearer_token_provider
from dotenv import load_dotenv

load_dotenv()
pool_management_endpoint = os.getenv("POOL_MANAGEMENT_ENDPOINT")
chat_endpoint=os.getenv("OPENAI_ENDPOINT")
model_deployment_name=os.getenv("OPENAI_DEPLOYMENT_NAME")

In [ ]:
async def handle_intermediate_steps(message: ChatMessageContent) -> None:
    for item in message.items or []:
        if isinstance(item, FunctionResultContent):
            print(f"# Function Result:> {item.result}")
        elif isinstance(item, FunctionCallContent):
            print(f"# Function Call:> {item.name} with arguments: {item.arguments}")
        else:
            print(f"# {message.name}: {message} ")

### Azure Dyamic Session Example

Authentication is also handled with service principals - just need to ensure the SPN has a role assignment to the resource with the `Azure ContainerApps Session Executor` role.

The example below configures a `ChatCompletionAgent` backed with a `SessionsPythonTool` to execute python in a sandboxed environment.  
There are a few things we can configure with the pool:
- Language type: Python
- Maximum concurrent sessions: 5
- Session cooldown period: 300 seconds
- Network egress: disabled


In [ ]:
import uuid
from httpx import AsyncClient
import tempfile

# 1. Create the python code interpreter tool using the SessionsPythonTool

# identifier = "c5930597-e370-4449-8601-13c93196e019"
# settings = SessionsPythonSettings(identifier=identifier)
python_code_interpreter = SessionsPythonTool(
    # settings=settings,
    pool_management_endpoint=pool_management_endpoint
)
        
async def create_agent():
    # 2. Create the agent
    agent = ChatCompletionAgent(
        service=AzureChatCompletion(
            endpoint=chat_endpoint,
            ad_token_provider=get_bearer_token_provider(EnvironmentCredential(), "https://cognitiveservices.azure.com/.default"),
            deployment_name=model_deployment_name
        ),
        name="MathTutor",
        instructions="You are helping a 10 year old boy with his math homework.  ALWAYS use the python code interpreter to solve the problem.",
        plugins=[SessionsPythonTool(
            # settings=settings,
            pool_management_endpoint=pool_management_endpoint
        )]
    )

    print("python_code_interpreter.settings.session_id", python_code_interpreter.settings.session_id)

    # # 3. Upload a CSV file to the session
    data = """Account,Year,Period
    100,2025,100
    100,2025,200
    100,2025,300
    100,2025,400
    100,2025,500
    100,2025,600
    100,2025,700
    """

    # file_name = ""
    # with tempfile.NamedTemporaryFile(delete=True) as temp:
    #     temp.write(data.encode("utf-8"))
    #     temp.flush()
    #     temp_path = temp.name
    #     file_metadata = await python_code_interpreter.upload_file(local_file_path=temp_path)
    #     file_name = file_metadata.filename

    TASK = f"Analyze the csv file {data} and add up all the values."
    print(f"# User: '{TASK}'")

    async for response in agent.invoke(
        messages=TASK,
        # on_intermediate_message=handle_intermediate_steps,
    ):
        print(f"# {response.name}: {response} ")
        
import asyncio

async def run_agent_instance(idx):
    print(f"Starting agent instance {idx}")
    await create_agent()
    print(f"Finished agent instance {idx}")

async def run_agents_in_parallel():
    tasks = [run_agent_instance(i) for i in range(10)]
    await asyncio.gather(*tasks)

# To actually run the agents in parallel, you would call:
await run_agents_in_parallel()
# await run_agent_instance(0)


Starting agent instance 0
python_code_interpreter.settings.session_id 7646c77d-9a2c-41f7-8d04-277ae5fa1b90
# User: 'Analyze the csv file Account,Year,Period
    100,2025,100
    100,2025,200
    100,2025,300
    100,2025,400
    100,2025,500
    100,2025,600
    100,2025,700
     and add up all the values.'
Starting agent instance 1
python_code_interpreter.settings.session_id 7646c77d-9a2c-41f7-8d04-277ae5fa1b90
# User: 'Analyze the csv file Account,Year,Period
    100,2025,100
    100,2025,200
    100,2025,300
    100,2025,400
    100,2025,500
    100,2025,600
    100,2025,700
     and add up all the values.'
Starting agent instance 2
python_code_interpreter.settings.session_id 7646c77d-9a2c-41f7-8d04-277ae5fa1b90
# User: 'Analyze the csv file Account,Year,Period
    100,2025,100
    100,2025,200
    100,2025,300
    100,2025,400
    100,2025,500
    100,2025,600
    100,2025,700
     and add up all the values.'
Starting agent instance 3
python_code_interpreter.settings.session_id 764

Function failed. Error: Code execution failed with status code 429 and error: {"$id":"1","code":"SessionsCountExceedsLimit","message":"The request sessions count is \u00271\u0027. There are already \u00275\u0027 active sessions and the maximum active sessions count is \u00275\u0027."}
Error invoking function SessionsPythonTool-execute_code: Code execution failed with status code 429 and error: {"$id":"1","code":"SessionsCountExceedsLimit","message":"The request sessions count is \u00271\u0027. There are already \u00275\u0027 active sessions and the maximum active sessions count is \u00275\u0027."}.
Traceback (most recent call last):
  File "/Users/kechan/code/bakeoff/.venv/lib/python3.10/site-packages/semantic_kernel/core_plugins/sessions_python_tool/sessions_python_plugin.py", line 201, in execute_code
    response.raise_for_status()
  File "/Users/kechan/code/bakeoff/.venv/lib/python3.10/site-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message,

# MathTutor: It looks like there was a technical problem analyzing the CSV file directly. But I can still add up the values for you step by step!

Here are the values from your table:

- Account: 100
- Year: 2025
- Periods: 100, 200, 300, 400, 500, 600, 700

Let's add them up:

100 + 200 + 300 + 400 + 500 + 600 + 700 = 2800

If you want the sum of all the numbers in the table, including both "Account", "Year", and "Period", let me know, and I'll add them all for you! Otherwise, the total for the "Period" column is 2800. 
Finished agent instance 8


Function failed. Error: Code execution failed with status code 429 and error: {"$id":"1","code":"SessionsCountExceedsLimit","message":"The request sessions count is \u00271\u0027. There are already \u00275\u0027 active sessions and the maximum active sessions count is \u00275\u0027."}
Error invoking function SessionsPythonTool-execute_code: Code execution failed with status code 429 and error: {"$id":"1","code":"SessionsCountExceedsLimit","message":"The request sessions count is \u00271\u0027. There are already \u00275\u0027 active sessions and the maximum active sessions count is \u00275\u0027."}.
Traceback (most recent call last):
  File "/Users/kechan/code/bakeoff/.venv/lib/python3.10/site-packages/semantic_kernel/core_plugins/sessions_python_tool/sessions_python_plugin.py", line 201, in execute_code
    response.raise_for_status()
  File "/Users/kechan/code/bakeoff/.venv/lib/python3.10/site-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message,

# MathTutor: It looks like there was a problem running the code. Let me explain how you can add up the numbers yourself!

Here's the list of the "Period" values from your csv:
- 100
- 200
- 300
- 400
- 500
- 600
- 700

Let's add them up step by step:

100 + 200 = 300  
300 + 300 = 600  
600 + 400 = 1000  
1000 + 500 = 1500  
1500 + 600 = 2100  
2100 + 700 = 2800  

So, the total sum of the "Period" values is 2800.

If you want, I can also show you how to do this with Python code! Would you like to see the code for that? 
Finished agent instance 9


Function failed. Error: Code execution failed with status code 429 and error: {"$id":"1","code":"SessionsCountExceedsLimit","message":"The request sessions count is \u00271\u0027. There are already \u00275\u0027 active sessions and the maximum active sessions count is \u00275\u0027."}
Error invoking function SessionsPythonTool-execute_code: Code execution failed with status code 429 and error: {"$id":"1","code":"SessionsCountExceedsLimit","message":"The request sessions count is \u00271\u0027. There are already \u00275\u0027 active sessions and the maximum active sessions count is \u00275\u0027."}.
Traceback (most recent call last):
  File "/Users/kechan/code/bakeoff/.venv/lib/python3.10/site-packages/semantic_kernel/core_plugins/sessions_python_tool/sessions_python_plugin.py", line 201, in execute_code
    response.raise_for_status()
  File "/Users/kechan/code/bakeoff/.venv/lib/python3.10/site-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message,

# MathTutor: It looks like there was an error with my code tool, so I couldn't add up the values directly.

But let's try to solve your problem step by step:

You have a table with three columns: Account, Year, and Period. The CSV data is:

Account,Year,Period  
100,2025,100  
100,2025,200  
100,2025,300  
100,2025,400  
100,2025,500  
100,2025,600  
100,2025,700  

Let's add up the values for each column:

For the "Account" column, all values are 100:
100 + 100 + 100 + 100 + 100 + 100 + 100 = 700

For the "Year" column, all values are 2025:
2025 + 2025 + 2025 + 2025 + 2025 + 2025 + 2025 = 14,175

For the "Period" column:
100 + 200 + 300 + 400 + 500 + 600 + 700 = 2,800

So, the totals are:
- Account total: 700
- Year total: 14,175
- Period total: 2,800

If you want me to show the steps or try again with the code when it's available, just let me know! 
Finished agent instance 4
# MathTutor: It looks like there was a technical problem and I wasn't able to run the code right now. Let me e

Function failed. Error: Code execution failed with status code 429 and error: {"$id":"1","code":"SessionsCountExceedsLimit","message":"The request sessions count is \u00271\u0027. There are already \u00275\u0027 active sessions and the maximum active sessions count is \u00275\u0027."}
Error invoking function SessionsPythonTool-execute_code: Code execution failed with status code 429 and error: {"$id":"1","code":"SessionsCountExceedsLimit","message":"The request sessions count is \u00271\u0027. There are already \u00275\u0027 active sessions and the maximum active sessions count is \u00275\u0027."}.
Traceback (most recent call last):
  File "/Users/kechan/code/bakeoff/.venv/lib/python3.10/site-packages/semantic_kernel/core_plugins/sessions_python_tool/sessions_python_plugin.py", line 201, in execute_code
    response.raise_for_status()
  File "/Users/kechan/code/bakeoff/.venv/lib/python3.10/site-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message,

# MathTutor: It looks like there is a technical issue with the code tool and I'm unable to run the code right now. 

But don't worry! I'll show you how to do this step by step:

Your table looks like this:

| Account | Year | Period |
|---------|------|--------|
| 100     | 2025 | 100    |
| 100     | 2025 | 200    |
| 100     | 2025 | 300    |
| 100     | 2025 | 400    |
| 100     | 2025 | 500    |
| 100     | 2025 | 600    |
| 100     | 2025 | 700    |

Now, let's add all the numbers in all columns (except the headers):

- First row: 100 + 2025 + 100 = 2225
- Second row: 100 + 2025 + 200 = 2325
- Third row: 100 + 2025 + 300 = 2425
- Fourth row: 100 + 2025 + 400 = 2525
- Fifth row: 100 + 2025 + 500 = 2625
- Sixth row: 100 + 2025 + 600 = 2725
- Seventh row: 100 + 2025 + 700 = 2825

Now, let's add all these together:

2225 + 2325 + 2425 + 2525 + 2625 + 2725 + 2825 = 17675

So the total sum of all the values is **17,675**. 
Finished agent instance 0
# MathTutor: It looks like I'm having 

: 

In [ ]:
import tempfile
import os

# Method 1: Using context manager (recommended)
def temp_file_example_context_manager():
    # This creates a temporary file that is automatically deleted when the context exits
    with tempfile.NamedTemporaryFile(delete=True) as temp:
        # Write some data to the temp file
        temp.write(b"Hello, temporary world!")
        temp.flush()  # Ensure data is written to disk
        
        # Get the file path
        temp_path = temp.name
        print(f"Temp file created at: {temp_path}")
        
        # You can read from the file
        temp.seek(0)
        content = temp.read()
        print(f"Content: {content}")
        
    # The file is automatically deleted when the with-block exits
    print(f"Does file still exist? {os.path.exists(temp_path)}")

# Method 2: Manual cleanup
def temp_file_example_manual():
    # Create a temporary file
    fd, temp_path = tempfile.mkstemp()
    try:
        # Write to the file using the file descriptor
        os.write(fd, b"Hello, temporary world!")
        
        print(f"Temp file created at: {temp_path}")
        
        # Read the file content
        with open(temp_path, 'rb') as f:
            content = f.read()
            print(f"Content: {content}")
            
    finally:
        # Close the file descriptor
        os.close(fd)
        # Delete the temporary file
        os.unlink(temp_path)
        
    print(f"Does file still exist? {os.path.exists(temp_path)}")

# Run the examples
print("Example 1: Using context manager")
temp_file_example_context_manager()

print("\nExample 2: Manual cleanup")
temp_file_example_manual()
